In [8]:
#Importamos los conectores para Snowflake y liberias necesarias
import snowflake.connector as sf
from snowflake.connector import DictCursor
import fxcmpy
import pandas as pd
#import datatime as dt

In [9]:
#Cadena de conexión Snowflake
username   = 'Blockchainassets'
mypassword = 'BonneNouvelle2020'
myaccount  = 'iw74759.eu-central-1'
warehouse  = 'COMPUTE_WH'
database   = 'DEMO_DB'

conn = sf.connect(user=username, password = mypassword, account = myaccount)
#Función que permite realizar la conexion a la nube de Snowflake y realizar procedimientos sql
def execute_query(connection, query):
    cursor = connection.cursor()
    cursor.execute(query)
    cursor.close()
    
#Conexión al WareHouse y el nombre de la Base de Datos a usar
try:
    sql = 'use {}'.format(database)
    execute_query(conn,sql)
    
    sql = 'use warehouse {}'.format(warehouse)
    execute_query(conn,sql)
    
    try:   
        sql = 'alter warehouse {} resume'.format(warehouse)
        execute_query(conn,sql)
    except: 
        pass
except Exception as e:
    print(e)

In [6]:
#Extracción de datos de FXCM
#Token para FXCM
con = fxcmpy.fxcmpy(access_token='e4191dcc0126d9c2559296e2501d74c5e1362395', log_level='error')
#Preparamos los datos y seran 50 registros de datos historicos (Candles)
data = con.get_candles('EUR/USD', period='m1', number=50)

#Se convierte Datetimeindex a formato normal de date time para la manipulacion de fechas
idx = data.index
idxtodt = pd.DataFrame({'month':idx})

In [ ]:
#Insert SQL a Snowflake
try:
    #--- Comienza una función para encapsular los procesos de insertar
    def function_insert():
        #Comienza la transacción
        startbeging = "begin name t4;"
        #Ejecutamos el Begin antes de realizar los insert into
        execute_query(conn, startbeging)
        #--- Comienza For (Ciclo para insertar 50 registros)
        for index in range(50):
            #Pasamos fecha, bidopen, bidclose
            sqldate     = idxtodt['month'].loc[index]  
            sqlbidopen  = data.iloc[index,0] 
            sqlbidclose = data.iloc[index,1]
            sql = "insert into DATA_CANDLES (DATE, BIDOPEN, BIDCLOSE) values ('{}','{}','{}')".format(sqldate, sqlbidopen, sqlbidclose)      
            execute_query(conn, sql)
        #--- Termina el FOR
        
        #Al terminar todo los insert ejecutamos el Commit para finalizar la transacción
        commit = "commit;"
        execute_query(conn,commit)
        print("Operación Exitosa")
    #--- Termina la función_insert 
    
    #Se ejecuta la función y realiza el proceso
    function_insert()
    
except:
    #En caso de una incidencia/riesgo se ejecuta el Rollback y se ejecutará de nuevo la función_insert
    rollback = "rollback;"
    execute_query(conn, rollback)
    print("Error, Operación Cancelada / Rollback / Se ejecutara de nuevo")
    function_insert()



Error, Operación Cancelada / Rollback / Se ejecutara de nuevo


In [ ]:
#Consultar datos
try:   
    #sql consulta
    sql = "select * from DATA_CANDLES"      
    cursor = conn.cursor(DictCursor)
    cursor.execute(sql)
    for c in cursor: 
        print(c)
        
except Exception as e:
    print(e)

 df = pd.read_sql(sql, conn)
   # print(df.iloc[0,0])
    #df.info()
    #print(df.head())